In [3]:
import utils
import pandas as pd
import time
from utils import get_secret 
from api_set import APIClient
from datetime import datetime
from pytz import timezone
import os
from datetime import datetime
from pytz import timezone
import asyncio

async def collect_keywords(srch_keyword, day):
    BASE_URL = get_secret("BASE_URL")
    CUSTOMER_ID = get_secret("CUSTOMER_ID")
    API_KEY = get_secret("API_KEY")
    SECRET_KEY = get_secret("SECRET_KEY")
    URI = get_secret("URI")
    METHOD = get_secret("METHOD")
    api_client = APIClient(BASE_URL, CUSTOMER_ID, API_KEY, SECRET_KEY, URI, METHOD)
    main_keyword = utils.load_keywords('main_keyword.json') 

    tasks = []
    
    for word in srch_keyword:
        if word in main_keyword:
            for keyword in main_keyword[word]:
                query = {
                    'siteld': '',
                    'bixtpld': '',
                    'event': '',
                    'month': '',
                    'showDetail': '1',
                    'hintKeywords': keyword
                }
                task = asyncio.create_task(api_client.get_data(query))
                tasks.append(task)
    
    responses = await asyncio.gather(*tasks)

    all_keywords_data = pd.DataFrame()
    for response in responses:
        if 'keywordList' in response:
            df = pd.DataFrame(response['keywordList'])
            df.replace('< 10', '9', inplace=True)
            columns_to_convert = ['monthlyPcQcCnt', 'monthlyMobileQcCnt']
            for column in columns_to_convert:
                df[column] = df[column].astype('float64')
            df['monthlyTotalCnt'] = df['monthlyPcQcCnt'] + df['monthlyMobileQcCnt']
            df = df[['relKeyword', 'monthlyTotalCnt']]  # 관심 있는 컬럼만 선택
            df.rename(columns={'relKeyword': '연관키워드', 'monthlyTotalCnt': '월간검색수_합계'}, inplace=True)  # 컬럼명 변경
            all_keywords_data = pd.concat([all_keywords_data, df], ignore_index=True)

    return all_keywords_data



async def main():
    today = datetime.now(timezone('Asia/Seoul'))
    day = today.strftime("%y%m%d")
    srch_keyword = ["keyword_final"]
    collected_data = await collect_keywords(srch_keyword, day)
    print(collected_data)

if __name__ == "__main__":
    loop = asyncio.get_event_loop()  # 현재 실행 중인 이벤트 루프를 가져옴
    if loop.is_running():  # 이벤트 루프가 이미 실행 중인지 확인
        # 이미 실행 중인 이벤트 루프에 작업을 등록
        task = loop.create_task(main())
    else:
        # 이벤트 루프가 실행 중이지 않은 경우, 비동기 함수를 실행
        loop.run_until_complete(main())

         연관키워드  월간검색수_합계
0           주식  528000.0
1          미주부     380.0
2        김현준대표     920.0
3         퀀트투자    6600.0
4         주식투자   12270.0
...        ...       ...
1983    투자조합출자     370.0
1984    재무제표강의     150.0
1985    저소득자대출     180.0
1986  벤처투자소득공제    2030.0
1987   소득공제형채권     320.0

[1988 rows x 2 columns]


In [3]:
from difflib import SequenceMatcher

# HTML 태그 제거 및 특수 문자 처리 함수
def clean_text(text):
    text = text.replace('<b>', ' ').replace('</b>', ' ')
    text = text.replace('&quot;', '"').replace('&apos;', '\'')
    text = text.replace('amp;', '').replace('&lt;', '<').replace('&gt;', '>')
    return text

# 검색어별 뉴스 제목, 본문, 링크 추출 및 필터링 함수
def news_result(names):
    titles, descs, links, index_list = [], [], [], []
    existing_titles = set()  # 기존 제목 관리용

    for name in names:
        json_data = news(name)
        if not json_data:  # 뉴스 데이터가 없는 경우 스킵
            continue

        tmp_titles, tmp_links = [], []
        count = min(len(json_data), 10)  # 최대 10개 항목 처리

        for i, item in enumerate(json_data):
            if i >= count:
                break

            # 뉴스 제목 필터링 및 처리
            title = clean_text(item['title'])
            if any(keyword in title for keyword in ['운세', '칼럼', '인사']) or title in existing_titles:
                continue  # 필터링 조건에 맞는 경우 스킵

            # 유사 제목 체크
            if any(SequenceMatcher(None, title, existing).ratio() > 0.8 for existing in existing_titles):
                continue

            # 제목, 본문, 링크 추가
            tmp_titles.append(title)
            existing_titles.add(title)  # 처리된 제목 추가
            descs.append(clean_text(item['description']))
            tmp_links.append(item['link'])
            index_list.append(len(tmp_titles))  # 인덱스 추가

        titles.append(tmp_titles)
        links.append(tmp_links)

    return titles, descs, links, index_list


import requests

def news(search_term, display=100):
    client_id = "ByXmMvAqMIxyVUY_h17L"
    client_secret = "2x7yByvNSN"
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    params = {
        "query": search_term,
        "display": display,
        "sort": "date"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()['items']
    else:
        print(f"Error fetching data for {search_term}: {response.status_code}")
        return []

# 예시 사용 방법
names = ["Python", "AI"]
results = news_result(names)
for result in results:
    print(result)

[['일반 사무직도 ‘코딩 능력’ 본다… 채용·승진 활용도 높아진 ‘코딩시험’', "엔드 투 엔드 데이터 프로세스 올인원 AI 플랫폼 '데이터이쿠 DSS'", '인투스카이·인천재능대, 2024 AI 드론 챌린지 대회 6~8일 개최', 'UNC5325 그룹, 이반티 제로데이 취약점 공격', "투썬캠퍼스 분당센터, '투썬 오픈스테이지 플러스' 성료", 'Surrealism Is 100. The World’s Still Surreal.', '“IT교육부터 취업 지원까지 한 번에…”', 'Keysight Introduces QuantumPro Delivering First Integrated EDA Workflow ...', '【채용 정보】 한화시스템, SPC삼립'], ['국립군산대, ‘2024 전북 산학관 커플링사업’ 선정', '설익은 XR기기 시장...조주완·저커버그는 반전 이룰까', '\'SK하이닉스 HBM 전문가가 美 마이크론으로?\'…업계 "엄벌해야"', '[계명대 소식] 계명대 대명캠퍼스 60년 만에 새모습으로 단장', '물러나는 정영채·김신… 증권사 CEO 세대교체 본격화', '추격당하는 삼성 스마트폰, 노태문 ‘1위 탈환’ 고심 커진다', "신동빈 '인공지능 전환' 주문에 부응하는 롯데건설, 계열사와 시너지도 기대", '민주당, "경기 남·동부 반도체 메가시티로 조성"', '네이버 오늘부터 ‘총선 페이지’ 열어… 허위 댓글 신고 기능 적용', " AI  날개 단 국내 1위 협업툴 '플로우'···업무 효율성 극대화"]]
["강여울 조선에듀 기자 | 디지털 기술의 중요성이 증가하면서 국내 기업들도 C, C++, C#, 자바(JAVA), 파이선( Python ) 등 컴퓨터 언어를 활용한 프로그래밍 '코딩(Coding)’ 시험을 채용∙승진 과정에 적극 활용하고 있다.... ", '대화형  Python , R, SQL 노트북을 통해 데이터를 시각적으로 탐색하고 분석 ③ 머신러닝 : 최첨단 머신러닝 라이브러리, AutoML,  Python , R을 통해 다양한 머

In [6]:
from difflib import SequenceMatcher

# HTML 태그 제거 및 특수 문자 처리 함수
def clean_text(text):
    text = text.replace('<b>', ' ').replace('</b>', ' ')
    text = text.replace('&quot;', '"').replace('&apos;', '\'')
    text = text.replace('amp;', '').replace('&lt;', '<').replace('&gt;', '>')
    return text

# 검색어별 뉴스 제목, 본문, 링크 추출 및 필터링 함수
def news_result(names):
    titles, descs, links, index_list = [], [], [], []
    existing_titles = set()  # 기존 제목 관리용

    for name in names:
        json_data = news(name)
        if not json_data:  # 뉴스 데이터가 없는 경우 스킵
            continue

        tmp_titles, tmp_links = [], []
        count = min(len(json_data), 11)  # 최대 10개 항목 처리

        for i, item in enumerate(json_data):
            if i >= count:
                break

            # 뉴스 제목 필터링 및 처리
            title = clean_text(item['title'])
            if any(keyword in title for keyword in ['운세', '칼럼', '인사']) or title in existing_titles:
                continue  # 필터링 조건에 맞는 경우 스킵

            # 유사 제목 체크
            if any(SequenceMatcher(None, title, existing).ratio() > 0.8 for existing in existing_titles):
                continue

            # 제목, 본문, 링크 추가
            tmp_titles.append(title)
            existing_titles.add(title)  # 처리된 제목 추가
            descs.append(clean_text(item['description']))
            tmp_links.append(item['link'])
            index_list.append(len(tmp_titles))  # 인덱스 추가

        titles.append(tmp_titles)
        links.append(tmp_links)

    return titles, descs, links, index_list


import requests

def news(search_term, display=100):
    client_id = "ByXmMvAqMIxyVUY_h17L"
    client_secret = "2x7yByvNSN"
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    params = {
        "query": search_term,
        "display": display,
        "sort": "date"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()['items']
    else:
        print(f"Error fetching data for {search_term}: {response.status_code}")
        return []

# 예시 사용 방법
names = ["Python", "AI"]
results = news_result(names)
for result in results:
    print(result)

[['[김정호의 반도체 특강] 1억원짜리라고? 엔비디아 반도체 경쟁력의 비밀은', '일반 사무직도 ‘코딩 능력’ 본다… 채용·승진 활용도 높아진 ‘코딩시험’', "엔드 투 엔드 데이터 프로세스 올인원 AI 플랫폼 '데이터이쿠 DSS'", '인투스카이·인천재능대, 2024 AI 드론 챌린지 대회 6~8일 개최', 'UNC5325 그룹, 이반티 제로데이 취약점 공격', "투썬캠퍼스 분당센터, '투썬 오픈스테이지 플러스' 성료", 'Surrealism Is 100. The World’s Still Surreal.', '“IT교육부터 취업 지원까지 한 번에…”', 'Keysight Introduces QuantumPro Delivering First Integrated EDA Workflow ...', '【채용 정보】 한화시스템, SPC삼립'], ['엔코아,  AI  제품·서비스 개발을 주도할 기술 인재 채용', '젠슨 황 엔비디아 CEO " AI  PC, 30년 만에 온 혁명"', "국민연금, 연금 수급자 900명 대상 '디지털·노후생활 교육' 지원", '[굿모닝 마켓] "금리 인하 머지않았다"…S&P500 사상 최고치 마감', '" AI  산업 생태계 만들자"…벤처기업협회-광주광역시 \'맞손\'', '삼성전자, 비스포크  AI  콤보 ‘새로운 런드리 라이프’ 체험 이벤트 진행', '게임사들의 IP 분쟁 격화…잇단 소송전 승자는?', '[클릭 e종목]“제룡전기, 전력 질주 초입 단계”', "삼성전자, '비스포크  AI  콤보' 체험 이벤트 진행", '3월 8일 美증시 특징주 [글로벌 시황&이슈]']]
['알고리즘은 파이선( Python ) 같은 코딩 언어로 표현되는데, 이 언어들은 인간 언어와 유사하다. 하지만 GPU 같은 반도체는 ‘1′과 ‘0′으로 이뤄진 2진수 언어인 ‘기계어’만 이해한다. 쿠다는 이렇게 인간... ', "강여울 조선에듀 기자 | 디지털 기술의 중요성이 증가하면서 국내 기업들도 C, C++, C#, 자바(JAVA), 파이선( Python ) 등

In [13]:
results[0]

[['[김정호의 반도체 특강] 1억원짜리라고? 엔비디아 반도체 경쟁력의 비밀은',
  '일반 사무직도 ‘코딩 능력’ 본다… 채용·승진 활용도 높아진 ‘코딩시험’',
  "엔드 투 엔드 데이터 프로세스 올인원 AI 플랫폼 '데이터이쿠 DSS'",
  '인투스카이·인천재능대, 2024 AI 드론 챌린지 대회 6~8일 개최',
  'UNC5325 그룹, 이반티 제로데이 취약점 공격',
  "투썬캠퍼스 분당센터, '투썬 오픈스테이지 플러스' 성료",
  'Surrealism Is 100. The World’s Still Surreal.',
  '“IT교육부터 취업 지원까지 한 번에…”',
  'Keysight Introduces QuantumPro Delivering First Integrated EDA Workflow ...',
  '【채용 정보】 한화시스템, SPC삼립'],
 ['엔코아,  AI  제품·서비스 개발을 주도할 기술 인재 채용',
  '젠슨 황 엔비디아 CEO " AI  PC, 30년 만에 온 혁명"',
  "국민연금, 연금 수급자 900명 대상 '디지털·노후생활 교육' 지원",
  '[굿모닝 마켓] "금리 인하 머지않았다"…S&P500 사상 최고치 마감',
  '" AI  산업 생태계 만들자"…벤처기업협회-광주광역시 \'맞손\'',
  '삼성전자, 비스포크  AI  콤보 ‘새로운 런드리 라이프’ 체험 이벤트 진행',
  '게임사들의 IP 분쟁 격화…잇단 소송전 승자는?',
  '[클릭 e종목]“제룡전기, 전력 질주 초입 단계”',
  "삼성전자, '비스포크  AI  콤보' 체험 이벤트 진행",
  '3월 8일 美증시 특징주 [글로벌 시황&이슈]']]

5